# RNN Play Generator
Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.

This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation

In [4]:
# %tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

# Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.

In [5]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Loading Your Own Data
To load your own data, you'll need to upload a file from the dialog below. Then you'll need to follow the steps from above but load in this new file instead.

In [6]:
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

##### this is optional

# Read Contents of File
Let's look at the contents of the file.

In [7]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [8]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Encoding
Since this text isn't encoded yet well need to do that ourselves. We are going to encode each unique character as a different integer.

In [9]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [10]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


And here we will make a function that can convert our numeric values to text.

In [11]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


# Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training examples we will prepapre will use a seq_length sequence as input and a seq_length sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

input: Hell | output: ello

Our first step will be to create a stream of characters from our text data.

In [12]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into input and output.

In [14]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [15]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches.

In [16]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.

In [18]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [19]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.6279061e-03  7.9170633e-03  2.1843887e-03 ... -2.0467909e-03
   -6.1178615e-04 -1.3653383e-03]
  [ 8.3141746e-03  6.0805185e-03 -1.0354999e-03 ... -1.4318120e-03
   -9.0850273e-04 -1.6672206e-03]
  [ 6.5625533e-03  5.7607945e-03 -1.2522059e-03 ... -3.1374558e-03
   -1.1063938e-03  2.9480811e-03]
  ...
  [-7.0370175e-03 -8.7125683e-03 -5.5741486e-03 ... -1.6262000e-03
   -4.8593390e-03  9.3167555e-03]
  [-7.9977242e-03 -6.1802403e-03 -8.7190187e-03 ... -1.7826462e-03
   -1.7546036e-03  6.5474226e-03]
  [-1.0851318e-02 -2.8635724e-03 -1.7513491e-03 ...  4.1644915e-04
    5.2153086e-03 -2.7861586e-04]]

 [[ 2.2232503e-04  1.5377239e-03 -4.9204228e-04 ... -1.6522498e-03
   -1.0769322e-03  4.5839208e-03]
  [-1.9995228e-03 -3.9446806e-03  8.2597625e-04 ...  1.6000052e-04
    2.1111662e-04 -4.7859410e-03]
  [-3.1928364e-03 -1.0758636e-03  2.2877543e-03 ...  3.2240404e-03
    3.2055657e-03 -1.7850912e-03]
  ...
  [-2.3946632e-03 -5.1015527e-03  3.4801345e-03 ...  2.6402376e

In [20]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00362791  0.00791706  0.00218439 ... -0.00204679 -0.00061179
  -0.00136534]
 [ 0.00831417  0.00608052 -0.0010355  ... -0.00143181 -0.0009085
  -0.00166722]
 [ 0.00656255  0.00576079 -0.00125221 ... -0.00313746 -0.00110639
   0.00294808]
 ...
 [-0.00703702 -0.00871257 -0.00557415 ... -0.0016262  -0.00485934
   0.00931676]
 [-0.00799772 -0.00618024 -0.00871902 ... -0.00178265 -0.0017546
   0.00654742]
 [-0.01085132 -0.00286357 -0.00175135 ...  0.00041645  0.00521531
  -0.00027862]], shape=(100, 65), dtype=float32)


In [21]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 3.6279061e-03  7.9170633e-03  2.1843887e-03  3.6516772e-03
 -3.9177178e-03  1.6706670e-04 -8.9728902e-04  5.4676121e-04
  3.0815017e-03  3.7797284e-03  5.6791245e-03  2.8217882e-03
 -5.2994490e-03  2.4391583e-03  2.7948129e-03  7.3519852e-03
  1.0171337e-03  1.2048427e-04  1.5602252e-03  3.9166338e-03
  1.1442471e-04 -1.5724477e-04 -3.7120387e-03  3.6333082e-03
  4.1363621e-03  8.4645776e-03 -1.7826976e-03  5.1188339e-03
 -9.0255926e-04  5.1824022e-03  1.3775757e-03 -5.0491584e-03
  3.0003404e-03 -8.2884904e-04 -4.5816028e-03 -7.2746020e-04
 -1.6156479e-03 -2.1841086e-03  2.5743889e-03 -2.8623063e-03
 -4.4493657e-03  1.7236670e-03 -2.1259682e-03  3.4497557e-03
 -2.1551996e-03  2.8082547e-03  1.0067609e-03  2.6969702e-03
  6.1042001e-03 -4.0530704e-04 -5.0141406e-03  1.5237698e-03
 -1.3077245e-03 -1.3604630e-03  1.6706450e-03  2.4575028e-03
 -1.9572864e-03 -4.7711088e-05  3.7136730e-03 -5.2340038e-04
  2.2765463e-03  2.4315037e-03 -2.0467909e-03 -6.1178615e-04
 -1.365338

In [22]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"cFl.wX-wY$NoI. !qBnDji!KlIlH?nW?lQx.P!\nMlA'?p3KqbAgVz,dQMMzBz'YIRNX3bq\njBcTXV,OKjQ&&\nh;pxsfGM.$!RnXL"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.

In [24]:
model.compile(optimizer='adam', loss=loss)

# Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Training
Finally, we will start training the model.

If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.

In [26]:
history = model.fit(data, epochs=1, callbacks=[checkpoint_callback])

172/172 [==============================] - 879s 5s/step - loss: 2.5476


# Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [27]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the lastest checkpoint that stores the models weights using the following line.

In [28]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

# #Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [33]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [34]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeop harcast:
Tout trich ane loncf parid to the musthes hith that bupyhase pyoud ouracy beaste's,
O ke nor of tles faatind baof unfy; ite bud,
To prowe th my thoud of on it is fom thengu:
Your thith:

nledk ID Hercem:
And count peat's it; my sher, ow the pisurby,
Thit ave thomed thougst main oor me kenceet
Norod a that ouncetmur, to bedveach a,
Thy layt! whot shatcin; ank wire are. you culs my wore
This carauntaim angey the ravegtoron, for!

Fronte nours, thou send cen sillawiom
And ceaftre kency Sinet cour to me, berist con Purty;
But ind Seach the, I bravy to siec allore willy soule tor ie thiek sainser,
That gouth anf waresan, gall buchind buthincomeer'e.

ELTENES:
Hom sinios wen, thin whe hapksour fror marest moor at thie with icans jourut you om scunment silt bust on Beil.

QUEE veand
Ha


In [35]:
print("holla")

holla
